In [137]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
grid_data_file = os.path.join(os.getcwd().split('data')[0], 'bi.pickle')
with open(grid_data_file,'rb') as f:
    load_array= pickle.load(f)
exc = [load_array[0][20], load_array[0][50]]
inh = [load_array[1][20], load_array[1][50]]

folder = str(os.getcwd().split('data_2')[1])
print(folder)
case = str(folder[1])

file_ = 'down.pickle'

c_var = [0,1]
if len(folder) == 4:
    if folder[3] == 'e':
        p_var = [0]
    elif folder[3] == 'i':
        p_var = [1]
else:
    p_var = [0,1]
    
print(p_var)

step1 = 100
step2 = 10
step3 = 1

/1_e
[0]


In [139]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [140]:
d_array = np.arange(500., 20, -0.1)
t0_array = d_array - 20.
t_pen = np.zeros(( t0_array.shape ))
for i in range(len(t_pen)):
    t_pen[i] = t0_array[i] / d_array[i]

In [141]:
dur_pre = 10
dur_post = 10

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-8
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

if case in ['1', '2']:    # low to high
    max_I = [3., -3.]
else:
    max_I = [-3., 3.]
    
if case in ['1', '3']:    # sparsity
    factor_ws = 1.
    factor_we = 0.
else:
    factor_ws = 0.
    factor_we = 1.
    
maxC = [1000., -1000., 0.18, 0.]
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [142]:
bestControl_ = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_ = np.array( [[None] * len(t_pen)] * len(exc) )
cost_ = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_ = np.array( [[None] * len(t_pen)] * len(exc) )
weights_ = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_ = np.array( [[None] * len(t_pen)] * len(exc) )

initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

print(bestControl_.shape)

(2, 4800)


In [143]:
if os.path.isfile(file_) :
    print("file found")
    
    with open(file_,'rb') as f:
        load_array = pickle.load(f)

    bestControl_ = load_array[0]
    bestState_ = load_array[1]
    cost_ = load_array[2]
    costnode_ = load_array[3]
    weights_ = load_array[4]

file found


In [144]:
# get initial parameters and target states

i_range = range(len(exc))
data.set_parameters(aln)
max_percent = []

i_range = [0]

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    aln.params.duration = 3000.
    
    control0 = aln.getZeroControl()
    control0 = functions.step_control(aln, maxI_ = max_I[0])

    aln.run(control=control0)
    
    target_rates = np.zeros((2))
    target_rates[0] = aln.rates_exc[0,-1] 
    target_rates[1] = aln.rates_inh[0,-1]

    control0 = functions.step_control(aln, maxI_ = max_I[1])
    aln.run(control=control0)

    init_state_vars = np.zeros(( len(state_vars) ))
    for j in range(len(state_vars)):
        if aln.state[state_vars[j]].size == 1:
            init_state_vars[j] = aln.state[state_vars[j]][0]
        else:
            init_state_vars[j] = aln.state[state_vars[j]][0,-1]

    initVars[i] = init_state_vars
    target[i] = [target_rates[0], target_rates[1]]
    
    max_percent_ind = 1
    if i == 1:
        max_percent_ind = 0
    
    max_percent.append( (np.argmax(np.abs(bestControl_[i][0]), axis=2)[0,max_percent_ind] -100) / 4800 )
    
print(max_percent)

-------  0 0.4500000000000001 0.4750000000000002
[0.5277083333333333]


In [145]:
# get uncontrolled cost

data.set_parameters(aln)
j = 0

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    dur = d_array[0]
    aln.params.duration = dur
    wp = 1. / (dur - t_pen[j] * dur)
    
    print(wp)
    
    target_ = aln.getZeroTarget()
    target_[:,0,:] = target[i][0]
    target_[:,1,:] = target[i][1]
            
    cost.setParams(wp, 0.0, 0.0)

    setinit(initVars[i], aln)
    control0 = aln.getZeroControl()

    # "HS", "FR", "PR", "HZ"
    cgv = None
    max_it = 0
    
    print(t_pen[0], dur, dur_pre, dur_post)

    bestControl_init_, bestState_init_, cost_init_, runtime_init_, grad_init_, phi_init_, phi1_, costnode_init_ = aln.A1(
        control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
        prec_variables_ = p_var, transition_time_ = t_pen[0])
    
    cost_uncontrolled[i] = cost_init_[0]

-------  0 0.4500000000000001 0.4750000000000002
0.05
set cost params:  0.05 0.0 0.0
0.96 500.0 10 10
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  0 , total integrated cost =  636.9058225135838
Improved over  0  iterations in  0.0  seconds by  0.0  percent.


In [114]:
i = 0

for j in range(4460,4800,1):
    convergence_[i][j] = None
    #bestControl_[i][j] = None

In [118]:
tol = 1e-6

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1

    while j < 4800:

        if j == 0:
            j += step
            continue
            
        if not convergence_[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)
        
        if j >= 3301:
            step = step2
        if j >=4791:
            step = step3

        if convergence_[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur
        
        wp = 1. / (dur - t_pen[j] * dur)

        cost.setParams(wp, 0., 1.)
        weights_[i][j] = cost.getParams()

        setinit(initVars[i], aln)

        max_it = 1000

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]
        
        max_percent_ind = 1
        if i == 1:
            max_percent_ind = 0

        if type(bestControl_[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,max_percent_ind] - 100) / (T-200)
            shift_ind = 0
            print(max_percent_prev, max_percent[i])
            if j < 4600:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_[i][j-step]), axis=2)[0,max_percent_ind] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1. * bestControl_[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_[i][j-step][:,:,100:-(100+step)]
                #control0[0,0,:] = 0.
        else:
            if bestControl_[i][j].shape[2] == T+200:
                control0 = bestControl_[i][j][:,:,100:-100]
                #control0[0,1,:] = 0.
            else:
                control0 = bestControl_[i][j][:,:,100:-(100+j)]

        bestControl_[i][j], bestState_[i][j], cost_[i][j], runtime_, grad_, phi_, phi1_, costnode_[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])
        
        time_ = np.arange(-dur_pre,round(dur+dur_post+aln.params.dt,1),aln.params.dt)
        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        print(costnode_[i][j][0][0][0], costnode_[i][j][2][0][:2])

        if cost_[i][j][-1] == 0.:
            convergence_[i][j] = True
            if j >= 3300:
                step = step2
            if j >=4790:
                step = step3

        with open(file_,'wb') as f:
            pickle.dump([bestControl_, bestState_, cost_, costnode_, weights_], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
----- 100
----- 200
----- 300
----- 400
----- 500
----- 600
----- 700
----- 800
----- 900
----- 1000
----- 1100
----- 1200
----- 1300
----- 1400
----- 1500
----- 1600
----- 1700
----- 1800
----- 1900
----- 2000
----- 2100
----- 2200
----- 2300
----- 2400
----- 2500
----- 2600
----- 2700
----- 2800
----- 2900
----- 3000
----- 3100
----- 3200
----- 3300
----- 3400
----- 3410
----- 3420
----- 3430
----- 3440
----- 3450
----- 3460
----- 3470
----- 3480
----- 3490
----- 3500
----- 3510
----- 3520
----- 3530
----- 3540
----- 3550
----- 3560
----- 3570
----- 3580
----- 3590
----- 3600
----- 3610
----- 3620
----- 3630
----- 3640
----- 3650
----- 3660
----- 3670
----- 3680
----- 3690
----- 3700
----- 3710
----- 3720
----- 3730
----- 3740
----- 3750
----- 3760
----- 3770
----- 3780
----- 3790
----- 3800
----- 3810
----- 3820
----- 3830
----- 3840
----- 3850
----- 3860
----- 3870
----- 3880
----- 3890
----- 3900
----- 3910
----- 3920
----- 3930
---

RUN  14 , total integrated cost =  19.900442138982335
RUN  15 , total integrated cost =  19.897942869596417
RUN  16 , total integrated cost =  19.895513027395765
RUN  17 , total integrated cost =  19.89307438941505
RUN  18 , total integrated cost =  19.890716631165336
RUN  19 , total integrated cost =  19.88840309832404
RUN  20 , total integrated cost =  19.88612229581576
RUN  30 , total integrated cost =  19.86444742192853
RUN  40 , total integrated cost =  19.844776073546047
RUN  50 , total integrated cost =  19.826500190939065
RUN  60 , total integrated cost =  19.810385793292173
RUN  70 , total integrated cost =  19.795817259521737
RUN  80 , total integrated cost =  19.7828098617729
RUN  90 , total integrated cost =  19.76984346714274
RUN  100 , total integrated cost =  19.756279926464945
RUN  110 , total integrated cost =  19.742813291523458
RUN  120 , total integrated cost =  19.730671860076818
RUN  130 , total integrated cost =  19.719315419251288
RUN  140 , total integrated cos

RUN  7 , total integrated cost =  19.496203760453575
RUN  8 , total integrated cost =  19.496203595934468
RUN  9 , total integrated cost =  19.496203246296893
RUN  10 , total integrated cost =  19.496202908418457
RUN  11 , total integrated cost =  19.496202741374155
RUN  12 , total integrated cost =  19.496202590187483
RUN  13 , total integrated cost =  19.496202224378063
RUN  14 , total integrated cost =  19.496201846501346
RUN  15 , total integrated cost =  19.49620170162713
RUN  16 , total integrated cost =  19.49620155336064
RUN  17 , total integrated cost =  19.49620120062271
RUN  18 , total integrated cost =  19.496200827354546
RUN  19 , total integrated cost =  19.496200699416708
RUN  20 , total integrated cost =  19.496200576815696
RUN  30 , total integrated cost =  19.496199639406743
RUN  40 , total integrated cost =  19.496198608410953
RUN  50 , total integrated cost =  19.496197605189916
RUN  60 , total integrated cost =  19.49619663459215
RUN  70 , total integrated cost =  

RUN  2 , total integrated cost =  19.496142801938728
RUN  3 , total integrated cost =  19.496142800843117
RUN  4 , total integrated cost =  19.496142799742206
RUN  5 , total integrated cost =  19.49614279880577
RUN  6 , total integrated cost =  19.496142797909368
RUN  7 , total integrated cost =  19.496142797020514
RUN  8 , total integrated cost =  19.496142796084197
RUN  9 , total integrated cost =  19.49614279499551
RUN  10 , total integrated cost =  19.496142793906966
RUN  11 , total integrated cost =  19.49614279297756
RUN  12 , total integrated cost =  19.49614279209027
RUN  13 , total integrated cost =  19.49614279120779
RUN  14 , total integrated cost =  19.496142790281542
RUN  15 , total integrated cost =  19.49614278919976
RUN  16 , total integrated cost =  19.49614278812343
RUN  17 , total integrated cost =  19.496142787200988
RUN  18 , total integrated cost =  19.49614278632272
RUN  19 , total integrated cost =  19.496142785446576
RUN  20 , total integrated cost =  19.496142

RUN  2 , total integrated cost =  20.702701854306223
RUN  3 , total integrated cost =  20.702681295325135
RUN  4 , total integrated cost =  20.70266076789474
RUN  5 , total integrated cost =  20.702640118587325
RUN  6 , total integrated cost =  20.70261892295127
RUN  7 , total integrated cost =  20.70259911809945
RUN  8 , total integrated cost =  20.70257952327219
RUN  9 , total integrated cost =  20.702560523946584
RUN  10 , total integrated cost =  20.702541674563342
RUN  11 , total integrated cost =  20.702522925793772
RUN  12 , total integrated cost =  20.702504105144893
RUN  13 , total integrated cost =  20.702484893240506
RUN  14 , total integrated cost =  20.70246551619529
RUN  15 , total integrated cost =  20.702445094226164
RUN  16 , total integrated cost =  20.70242511906988
RUN  17 , total integrated cost =  20.702405639461563
RUN  18 , total integrated cost =  20.70238608388334
RUN  19 , total integrated cost =  20.702366494372278
RUN  20 , total integrated cost =  20.70234

...

RUN  14 , total integrated cost =  31.68612427548656
RUN  15 , total integrated cost =  31.68612425116023
RUN  16 , total integrated cost =  31.686124227014446
RUN  17 , total integrated cost =  31.686124202746996
RUN  18 , total integrated cost =  31.686124178633406
RUN  19 , total integrated cost =  31.6861241544638
RUN  20 , total integrated cost =  31.686124130381405
RUN  30 , total integrated cost =  31.68612389105094
RUN  40 , total integrated cost =  31.686123654789697
RUN  50 , total integrated cost =  31.686123421461254
RUN  60 , total integrated cost =  31.68612319115616
RUN  70 , total integrated cost =  31.686122963670943
RUN  80 , total integrated cost =  31.68612273916314
RUN  90 , total integrated cost =  31.686122517366954
RUN  100 , total integrated cost =  31.686122298502188
RUN  110 , total integrated cost =  31.686122082245202
RUN  120 , total integrated cost =  31.686121868854947
RUN  130 , total integrated cost =  31.686121658010375
RUN  140 , total integrated cos

RUN  7 , total integrated cost =  31.686105098357128
RUN  8 , total integrated cost =  31.686105097558958
RUN  9 , total integrated cost =  31.686105096790392
RUN  10 , total integrated cost =  31.686105096081974
RUN  11 , total integrated cost =  31.686105095405573
RUN  12 , total integrated cost =  31.68610509478147
RUN  13 , total integrated cost =  31.68610509416534
RUN  14 , total integrated cost =  31.68610509355869
RUN  15 , total integrated cost =  31.6861050929521
RUN  16 , total integrated cost =  31.686105092339524
RUN  17 , total integrated cost =  31.68610509171912
RUN  18 , total integrated cost =  31.68610509105096
RUN  19 , total integrated cost =  31.68610509034649
RUN  20 , total integrated cost =  31.68610508959359
RUN  30 , total integrated cost =  31.686105082340813
RUN  40 , total integrated cost =  31.686105076028372
RUN  50 , total integrated cost =  31.68610506897228
RUN  60 , total integrated cost =  31.686105062830972
RUN  70 , total integrated cost =  31.686

RUN  6 , total integrated cost =  34.848554026945465
RUN  7 , total integrated cost =  34.84855392664735
RUN  8 , total integrated cost =  34.8485538241334
RUN  9 , total integrated cost =  34.84855372580938
RUN  10 , total integrated cost =  34.84855362203098
RUN  11 , total integrated cost =  34.84855352492605
RUN  12 , total integrated cost =  34.8485534258016
RUN  13 , total integrated cost =  34.84855332518141
RUN  14 , total integrated cost =  34.84855322494046
RUN  15 , total integrated cost =  34.84855312620797
RUN  16 , total integrated cost =  34.84855302477993
RUN  17 , total integrated cost =  34.8485529275707
RUN  18 , total integrated cost =  34.84855283055923
RUN  19 , total integrated cost =  34.84855272968838
RUN  20 , total integrated cost =  34.84855263162986
RUN  30 , total integrated cost =  34.8485516521524
RUN  40 , total integrated cost =  34.848550688645595
RUN  50 , total integrated cost =  34.84854974570943
RUN  60 , total integrated cost =  34.84854881305782

RUN  2 , total integrated cost =  34.84848879616561
RUN  3 , total integrated cost =  34.84848879477986
RUN  4 , total integrated cost =  34.848488793372795
RUN  5 , total integrated cost =  34.848488791935374
RUN  6 , total integrated cost =  34.848488790511745
RUN  7 , total integrated cost =  34.84848878909999
RUN  8 , total integrated cost =  34.84848878765878
RUN  9 , total integrated cost =  34.848488786273734
RUN  10 , total integrated cost =  34.84848878481391
RUN  11 , total integrated cost =  34.84848878344534
RUN  12 , total integrated cost =  34.84848878205001
RUN  13 , total integrated cost =  34.848488780630746
RUN  14 , total integrated cost =  34.84848877921888
RUN  15 , total integrated cost =  34.848488777825175
RUN  16 , total integrated cost =  34.84848877639576
RUN  17 , total integrated cost =  34.84848877502867
RUN  18 , total integrated cost =  34.84848877358064
RUN  19 , total integrated cost =  34.848488772229054
RUN  20 , total integrated cost =  34.848488770

IndexError: index 4800 is out of bounds for axis 0 with size 4800

In [146]:
bestControl_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
bestState_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
cost_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
costnode_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
weights_maxW = np.array( [[None] * len(t_pen)] * len(exc) )
convergence_maxW = np.array( [[None] * len(t_pen)] * len(exc) )

file_maxW = 'down_Wmax.pickle'

In [147]:
if os.path.isfile(file_maxW) :
    print("file found")
    
    with open(file_maxW,'rb') as f:
        load_array = pickle.load(f)

    bestControl_maxW = load_array[0]
    bestState_maxW = load_array[1]
    cost_maxW = load_array[2]
    costnode_maxW = load_array[3]
    weights_maxW = load_array[4]

file found


In [121]:
tol = 1e-8

for i in i_range:
    print('------------', i)
    j = 0

    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.

    aln.params.duration = dur

    while not convergence_maxW[i][j]:
        dur = d_array[j]
        aln.params.duration = dur
        wp = 1. / (dur - t_pen[j] * dur)

        setinit(initVars[i], aln)

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        control0 = aln.getZeroControl()
        if type(bestControl_maxW[i][j]) == type(None):
            max_it = 10
            control0 = bestControl_[i][0][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i]
                       - wp * sum(costnode_[i][j][0][0][:]) ) / sum( costnode_[i][j][2][0][:] )
            cost.setParams(wp, weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
        else:
            control0 = bestControl_maxW[i][j][:,:,n_pre-1:-n_post+1]
            weight_ = ( cost_uncontrolled[i]
                       - wp * sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            cost.setParams(wp, weight_ * factor_we, weight_ * factor_ws)
            weights_maxW[i][j] = cost.getParams()
            max_it = int(100)

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True

        print('cost splitting : ', costnode_maxW[i][j][0][0][0], costnode_maxW[i][j][2][0][:2])

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

------------ 0
set cost params:  0.05 0.0 190.47003207793207
interpolate adjoint :  True True True
initial cost =  634.4855803880291
RUN  0 , total integrated cost =  634.4855803880291
Gradient descend method:  None
RUN  1 , total integrated cost =  634.4855803880291
Control only changes marginally.
RUN  1 , total integrated cost =  634.4855803880291
Improved over  1  iterations in  0.5308340739866253  seconds by  0.0  percent.
cost splitting :  2.2385916167652766e-05 [1.25115182e-15 3.33115699e+00]


In [148]:
i=0
for j in range(2600,3000,1):
    convergence_maxW[i][j] = None
    #bestControl_maxW[i][j] = None

In [149]:
tol = 1e-5

for i in i_range:
    print("------- ", i, exc[i], inh[i])
    aln.params.ext_exc_current = exc[i] * 5.
    aln.params.ext_inh_current = inh[i] * 5.
    
    j = 0
    step = step1

    k = 0
    
    ind_max_c = 1
    if i == 1:
        ind_max_c = 0

    while j < 4800:

        if j == 0:
            j += step
            continue
            
        if not convergence_maxW[i][j-step]:
            if j - step > 0:
                j -= step
                print('go back to -----', j)

        print('-----', j)
        
        if j >= 3001:
            step = step2
        if j >=4790:
            step = step3

        if convergence_maxW[i][j]:
            j += step
            continue

        dur = d_array[j]
        T = 5001 - j
        aln.params.duration = dur
        setinit(initVars[i], aln)
        wp = 1. / (dur - t_pen[j] * dur)

        max_it = 1000

        target_ = aln.getZeroTarget()
        target_[:,0,:] = target[i][0]
        target_[:,1,:] = target[i][1]

        if type(bestControl_maxW[i][j]) == type(None):
            max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,ind_max_c] - 100) / (T-200)
            shift_ind = 0
            print(max_percent_prev, max_percent[i])
            if j < 4000:
                while max_percent_prev > max_percent[i]:
                    shift_ind += 1
                    max_percent_prev = (np.argmax(np.abs(bestControl_maxW[i][j-step]), axis=2)[0,ind_max_c] - 100 - shift_ind) / (T-200)

            control0 = np.zeros(( bestControl_maxW[i][j-step][:,:,100:-(100+step)].shape ))
            print('shift ind = ', shift_ind)
            if shift_ind != 0:
                control0[:,:,:-shift_ind] = 1. * bestControl_maxW[i][j-step][:,:,100+shift_ind:-(100+step)]
            else:
                control0[:,:,:] = 1. * bestControl_maxW[i][j-step][:,:,100:-(100+step)]
                
            weight_ = ( cost_uncontrolled[i]
                          - wp * sum(costnode_maxW[i][j-step][0][0][:]) ) / sum( costnode_maxW[i][j-step][2][0][:] )
        
        else:
            if bestControl_maxW[i][j].shape[2] == T+200:
                control0 = bestControl_maxW[i][j][:,:,100:-100]
                weight_ = ( cost_uncontrolled[i]
                           - wp * sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
            else:
                control0 = bestControl_maxW[i][j][:,:,100:-(100+j)]
                
                weight_ = ( cost_uncontrolled[i]
                           - wp * sum(costnode_maxW[i][j][0][0][:]) ) / sum( costnode_maxW[i][j][2][0][:] )
             
        """
        if j >= 3800:
            if k == 0:
                weight_ *= 0.6
                max_it = 100
            elif k < 3:
                weight_ *= 0.8
                max_it = 100
            elif k < 5:
                weight_ *= 0.9
                max_it = 100
        """
            
            
        cost.setParams(wp, weight_ * factor_we, weight_ * factor_ws)
        weights_maxW[i][j] = cost.getParams()

        bestControl_maxW[i][j], bestState_maxW[i][j], cost_maxW[i][j], runtime_, grad_, phi_, phi1_, costnode_maxW[i][j] = aln.A1(
            control0, target_, c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
            startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
            t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = c_var,
            prec_variables_ = p_var, transition_time_ = t_pen[j])
        
        k +=1

        print(costnode_maxW[i][j][0][0][:2], costnode_maxW[i][j][2][0][:2])

        if cost_maxW[i][j][2] == 0.:
            convergence_maxW[i][j] = True
            if j >= 3000:
                step = step2
            if j >=4790:
                step = step3

        with open(file_maxW,'wb') as f:
            pickle.dump([bestControl_maxW, bestState_maxW, cost_maxW, costnode_maxW, weights_maxW], f)

        j += step

-------  0 0.4500000000000001 0.4750000000000002
----- 100
set cost params:  0.05 0.0 191.1965763642886
interpolate adjoint :  True True True
initial cost =  636.9058225135836
RUN  0 , total integrated cost =  636.9058225135836
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135836
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135836
Improved over  1  iterations in  0.7004767910111696  seconds by  0.0  percent.
[0.00020332 0.        ] [1.25115182e-15 3.33115699e+00]
----- 200
set cost params:  0.05 0.0 191.1965520067019
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.5088881049887277  seconds by  0.0  percent.
[0.00182615 0.        ] [1.25115181e-15 3.3

----- 1800
set cost params:  0.05 0.0 191.19279357341978
interpolate adjoint :  True True True
initial cost =  636.9058225135836
RUN  0 , total integrated cost =  636.9058225135836
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135836
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135836
Improved over  1  iterations in  0.3129170009633526  seconds by  0.0  percent.
[0.02439946 0.        ] [5.96583793e-05 3.33115691e+00]
----- 1900
set cost params:  0.05 0.0 191.18990049032143
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.3009931549895555  seconds by  0.0  percent.
[0.21734363 0.        ] [5.96583774e-05 3.33115686e+00]
----- 2000
set cost params:  0.0

----- 3050
set cost params:  0.05 0.0 190.939722303202
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.2796362020308152  seconds by  0.0  percent.
[2.92581769 0.        ] [5.96569273e-05 3.33481233e+00]
----- 3060
set cost params:  0.05 0.0 190.93184736929868
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.27986712602432817  seconds by  0.0  percent.
[2.98570913 0.        ] [5.96568425e-05 3.33493419e+00]
----- 3070
set cost params:  0.05

----- 3220
set cost params:  0.05 0.0 190.7278648560743
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.2572720879688859  seconds by  0.0  percent.
[5.26650392 0.        ] [5.96549788e-05 3.33790304e+00]
----- 3230
set cost params:  0.05 0.0 190.709906696719
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.2546528049861081  seconds by  0.0  percent.
[5.61628671 0.        ] [5.96548142e-05 3.33812565e+00]
----- 3240
set cost params:  0.05 0

----- 3380
set cost params:  0.05 0.0 190.37220247293553
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.1383865859825164  seconds by  0.0  percent.
[9.46719513 0.        ] [5.9650927e-05 3.3430359e+00]
----- 3390
set cost params:  0.05 0.0 190.340666611639
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135838
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135838
Improved over  1  iterations in  0.2355645999778062  seconds by  0.0  percent.
[9.78974772 0.        ] [5.96505664e-05 3.34350506e+00]
----- 3400
set cost params:  0.05 0.

...

----- 4520
set cost params:  0.05 0.0 157.10778604904775
interpolate adjoint :  True True True
initial cost =  636.9058225135836
RUN  0 , total integrated cost =  636.9058225135836
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9056571467776
RUN  2 , total integrated cost =  636.9056571467776
Control only changes marginally.
RUN  2 , total integrated cost =  636.9056571467776
Improved over  2  iterations in  0.1771853449754417  seconds by  2.5964090795582706e-05  percent.
[323.98988319   0.        ] [1.32445278e-05 3.95081681e+00]
go back to ----- 4520
----- 4520
set cost params:  0.05 0.0 157.1078279052663
interpolate adjoint :  True True True
initial cost =  636.905822513584
RUN  0 , total integrated cost =  636.905822513584
Gradient descend method:  None
RUN  1 , total integrated cost =  636.905822513584
Control only changes marginally.
RUN  1 , total integrated cost =  636.905822513584
Improved over  1  iterations in  0.08687476499471813  seconds by  0.0  perc

----- 4590
set cost params:  0.05 0.0 145.04704353619135
interpolate adjoint :  True True True
initial cost =  636.9058225135838
RUN  0 , total integrated cost =  636.9058225135838
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9056120704204
RUN  2 , total integrated cost =  636.9056087622001
RUN  3 , total integrated cost =  636.9056062405521
RUN  4 , total integrated cost =  636.9055606041748
RUN  5 , total integrated cost =  636.9055478830426
RUN  6 , total integrated cost =  636.9055429320323
RUN  7 , total integrated cost =  636.9055321337204
RUN  8 , total integrated cost =  636.9055309328822
RUN  9 , total integrated cost =  636.9055305571426
RUN  10 , total integrated cost =  636.9055163967546
RUN  11 , total integrated cost =  636.905502987339
RUN  12 , total integrated cost =  636.905502987339
Control only changes marginally.
RUN  12 , total integrated cost =  636.905502987339
Improved over  12  iterations in  1.400175285001751  seconds by  5.01685231171

----- 4670
set cost params:  0.05 0.0 110.92629231908676
interpolate adjoint :  True True True
initial cost =  636.9058225135839
RUN  0 , total integrated cost =  636.9058225135839
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135839
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135839
Improved over  1  iterations in  0.08467163302702829  seconds by  0.0  percent.
[1329.77292999    0.        ] [1.28281306 3.85949508]
----- 4680
set cost params:  0.05 0.0 105.94309822617834
interpolate adjoint :  True True True
initial cost =  636.9058225135836
RUN  0 , total integrated cost =  636.9058225135836
Gradient descend method:  None
RUN  1 , total integrated cost =  636.9058225135836
Control only changes marginally.
RUN  1 , total integrated cost =  636.9058225135836
Improved over  1  iterations in  0.07685580698307604  seconds by  0.0  percent.
[1395.75749716    0.        ] [1.76208671 3.59095617]
----- 4690
set cost params:  0.05 

----- 4793
set cost params:  0.05 0.0 60.91941969020988
interpolate adjoint :  True True True
initial cost =  636.9058225135835
RUN  0 , total integrated cost =  636.9058225135835
Gradient descend method:  None
RUN  1 , total integrated cost =  636.905270142795
RUN  2 , total integrated cost =  636.9049350590863
RUN  3 , total integrated cost =  636.9046340896982
RUN  4 , total integrated cost =  636.904186345049
RUN  5 , total integrated cost =  636.903724244027
RUN  6 , total integrated cost =  636.9033020580503
RUN  7 , total integrated cost =  636.902832210006
RUN  8 , total integrated cost =  636.9023765999423
RUN  9 , total integrated cost =  636.9019156035229
RUN  10 , total integrated cost =  636.9014167766067
RUN  11 , total integrated cost =  636.9011622694753
RUN  12 , total integrated cost =  636.9008800315323
RUN  13 , total integrated cost =  636.9005839152609
RUN  14 , total integrated cost =  636.9002169488577
RUN  15 , total integrated cost =  636.8998010000768
RUN  16

RUN  12 , total integrated cost =  636.8948415154812
RUN  13 , total integrated cost =  636.8940767547259
RUN  14 , total integrated cost =  636.8931888088264
RUN  15 , total integrated cost =  636.8924362857624
RUN  16 , total integrated cost =  636.8921468706517
RUN  17 , total integrated cost =  636.8918949559245
RUN  18 , total integrated cost =  636.8917403296723
RUN  19 , total integrated cost =  636.8916316121154
RUN  20 , total integrated cost =  636.8916239540321
RUN  30 , total integrated cost =  636.8910195093828
RUN  40 , total integrated cost =  636.8901121839342
Control only changes marginally.
RUN  42 , total integrated cost =  636.8901103480644
Improved over  42  iterations in  2.899564000952523  seconds by  0.002466952721121629  percent.
[3486.13270816    0.        ] [7.90892414e+00 2.15074997e-06]
